In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import anndata
from pybedtools import BedTool

## Parameter

In [2]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial/ITSpatial_rms_results_collapsed.tsv'
dmr_prefix = 'DGmCH'

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
skip_quantile = 0.25
delta_to_mean = 0.1
dms_cutoff = 1

In [3]:
# Parameters
dmr_path = "/home/hanliu/project/mouse_rostral_brain/DMR/SubType/raw/SubType-chr10/SubType-chr10_rms_results_collapsed.tsv"
dmr_prefix = "Sub"
black_list_path = "/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz"
skip_quantile = 0.25
delta_to_mean = 0.3
dms_cutoff = 1


## Read DMR

In [4]:
# get column and n_sample
dmr_column = pd.read_csv(dmr_path, sep='\t', nrows=0).columns
n_sample = dmr_column.size - 6
dmr_column = dmr_column.map(lambda i: i[18:] if 'methylation_level_' in i else i)
samples = dmr_column[6:]

dmr_region_infos = []
mc_rates = []
hypo_sig_dict = {sample: [] for sample in samples}
hyper_sig_dict = {sample: [] for sample in samples}

dmr_df = pd.read_csv(dmr_path, sep='\t', skiprows=1, names=dmr_column)
print(dmr_df.shape[0], 'dmr before filter')
dmr_df.index = dmr_df.index.map(lambda i: f'{dmr_prefix}_{i}')

# save raw info
dmr_bed = dmr_df.iloc[:, :4].copy()
mc_rate = dmr_df.iloc[:, 6:].copy()

244060 dmr before filter


## Save unfiltered dmr info

In [5]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    hdf['Rate'] = mc_rate
    hdf['Bed'] = dmr_bed

In [6]:
dmr_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(pathlib.Path(dmr_path).parent / 'TotalDMR.nofilter.bed', 
                                                   index=None, header=None, sep='\t')

## Filter DMR based on min-max delta and blacklist

In [7]:
# filter by min max delta first, if DMR do not pass min max filter, it won't have sig sample anyway
delta_judge = (mc_rate.max(axis=1) - mc_rate.min(axis=1)) > delta_to_mean
delta_index = mc_rate.index[delta_judge]

print(delta_index.size, 'dmr pass delta filter')

227634 dmr pass delta filter


In [8]:
# filter by blacklist
black_list = pd.read_csv(black_list_path, sep='\t', header=None)
black_list = BedTool.from_dataframe(black_list)

_dmr_bed = BedTool.from_dataframe(dmr_bed.reset_index().iloc[:, [1,2,3,0]])
filtered_dmr_bed = _dmr_bed.intersect(black_list, v=True).to_dataframe().set_index('name')

white_index = filtered_dmr_bed.index
print(white_index.size, 'dmr pass blacklist filter')

237777 dmr pass blacklist filter


In [9]:
dmr_df = dmr_df.loc[delta_index & white_index]
dmr_df = dmr_df[dmr_df['number_of_dms'] >= dms_cutoff]
print(dmr_df.shape[0], 'dmr after filter')
dmr_df.head()

221799 dmr after filter


,#chr,start,end,number_of_dms,hypermethylated_samples,hypomethylated_samples,CT-L6_Il1rap,NP-L6_Cntnap5a,CGE-Lamp5_Sorcs1,CGE-Vip_Grm8,...,Foxp2_Trpc7,CGE-Lamp5_Grk5,IT-L5_Cdh8,IG-CA2_Peak1,ANP_anp-dg,OLF_Xkr6,D1L-Fstl4_Cadm1,IT-L4_Shc3,D1L-PAL_Plcxd3,L6b_Pkhd1
Sub_46,chr10,3136346,3136910,6,"CA3_Cadm2,L6b_Adcy8,PT-L5_Ptprt,OLF-Exc_Cux2,N...","VLMC_Col4a1,D1L-Fstl4_Crim1,MGC_mgc-all,ODC_od...",0.888283,0.920000,0.777778,0.842105,...,0.841935,0.864865,0.857567,0.886957,0.849057,0.928571,0.827815,0.872024,0.822917,0.84
Sub_47,chr10,3137839,3137942,2,"PAL-Inh_Onecut2,PT-L5_Ptprt,NP-L6_Kcnab1,CT-L6...","ASC_cortex-olf,VLMC_Col4a1,DG_dg-all,MGC_mgc-a...",0.796117,0.866667,0.781250,0.761905,...,0.709677,0.723404,0.804348,0.720930,0.434783,1.000000,0.654545,0.817460,0.729167,1.00
Sub_48,chr10,3138567,3138843,5,"PT-L5_Kcnh1,L6b_Nrp2,IT-L5_Etv1,PT-L5_Ptprt,IT...","ASC_cortex-olf,DG_dg-all,ODC_odc-small,CA3_Cad...",0.876494,0.931818,0.883721,0.800000,...,0.812500,0.871658,0.882927,0.831579,0.507937,0.909091,0.828571,0.851852,0.846847,1.00
Sub_49,chr10,3138863,3138863,1,NaN,"VLMC_Col4a1,L6b_Nrp2,CT-L6_Hcrtr2,PT-L5_Plcb4,...",0.939394,1.000000,0.777778,1.000000,...,0.925000,0.938776,0.941176,1.000000,0.923077,1.000000,0.933333,1.000000,0.730769,NaN
Sub_50,chr10,3139118,3139118,1,"PT-L5_Tenm2,PT-L5_Kcnh1,IG-CA2_Chrm3,PAL-Inh_O...","MGC_mgc-all,ODC_odc-small,IT-L23_Ptprt,MSN-D1_...",0.466667,0.400000,0.200000,0.500000,...,0.538462,0.311111,0.500000,0.600000,0.428571,NaN,0.350000,0.423077,0.294118,NaN


## Calculate sample delta to robust mean

In [10]:
mc_rate = mc_rate.loc[dmr_df.index].copy()
n_sample = mc_rate.shape[1]

assert skip_quantile < 0.5
skip_n = min(1, round(n_sample * skip_quantile))

In [11]:
# ~1M rows / min, depending on n_sample, not very slow
def robust_mean(row):
    return np.nanmean(np.sort(row)[skip_n:-skip_n])
robust_mean = np.apply_along_axis(robust_mean, 1, mc_rate.values)
robust_mean = pd.Series(robust_mean, index=mc_rate.index)

In [12]:
delta = mc_rate - robust_mean[:, None]
# 0 is hypo DMR, 1 is hyper DMR, nan is not significant
judge = (delta > 0)[delta.abs() > delta_to_mean]

In [13]:
robust_mean_hypo_records = {}
robust_mean_hyper_records = {}

for sample, sample_col in judge.iteritems():
    hypo_dmr = sample_col[sample_col == 0].index
    hyper_dmr = sample_col[sample_col == 1].index
    
    robust_mean_hypo_records[sample] = hypo_dmr
    robust_mean_hyper_records[sample] = hyper_dmr

## Assign DMR to each sample

In [14]:
# parse results and add the robust mean judge
for sample in samples:
    # here saved a list of np.array
    hypo_index = dmr_df[dmr_df['hypomethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hyper_index = dmr_df[dmr_df['hypermethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hypo_sig_dict[sample] = hypo_index & robust_mean_hypo_records[sample]
    hyper_sig_dict[sample] = hyper_index & robust_mean_hyper_records[sample]
    print(sample, 'HypoDMR ', hypo_sig_dict[sample].size, sep='\t')
    print(sample, 'HyperDMR', hyper_sig_dict[sample].size, sep='\t')


CT-L6_Il1rap	HypoDMR 	16210
CT-L6_Il1rap	HyperDMR	2591
NP-L6_Cntnap5a	HypoDMR 	6422
NP-L6_Cntnap5a	HyperDMR	10300
CGE-Lamp5_Sorcs1	HypoDMR 	14483
CGE-Lamp5_Sorcs1	HyperDMR	5664
CGE-Vip_Grm8	HypoDMR 	8956
CGE-Vip_Grm8	HyperDMR	6314
LSX-Inh_Dock10	HypoDMR 	4490
LSX-Inh_Dock10	HyperDMR	10917
CGE-Vip_Ccser1	HypoDMR 	8196
CGE-Vip_Ccser1	HyperDMR	8631
CGE-Vip_Ntng1	HypoDMR 	6827
CGE-Vip_Ntng1	HyperDMR	3498
ASC_cortex-olf	HypoDMR 	18277
ASC_cortex-olf	HyperDMR	9177
MGE-Sst_Chodl	HypoDMR 	6899
MGE-Sst_Chodl	HyperDMR	7704
IT-L6_Oxr1	HypoDMR 	17048
IT-L6_Oxr1	HyperDMR	2683
VLMC_Col4a1	HypoDMR 	32869
VLMC_Col4a1	HyperDMR	8445
LSX-Inh_Lats2	HypoDMR 	13488
LSX-Inh_Lats2	HyperDMR	5430
L6b_Kcnk2	HypoDMR 	15262
L6b_Kcnk2	HyperDMR	5481
Chd7_Megf11	HypoDMR 	9673
Chd7_Megf11	HyperDMR	7403
MGE-Sst_Bmper	HypoDMR 	5367
MGE-Sst_Bmper	HyperDMR	10432
CT-L6_Megf9	HypoDMR 	15174
CT-L6_Megf9	HyperDMR	2388
Chd7_Kcnc2	HypoDMR 	4755
Chd7_Kcnc2	HyperDMR	5187
DG-po_Kctd8	HypoDMR 	12778
DG-po_Kctd8	HyperDMR	1312
DG_dg-

OLF_Mapk10	HypoDMR 	8849
OLF_Mapk10	HyperDMR	2362
Foxp2_Trpc7	HypoDMR 	14299
Foxp2_Trpc7	HyperDMR	6042
CGE-Lamp5_Grk5	HypoDMR 	21916
CGE-Lamp5_Grk5	HyperDMR	3294
IT-L5_Cdh8	HypoDMR 	26342
IT-L5_Cdh8	HyperDMR	1104
IG-CA2_Peak1	HypoDMR 	31249
IG-CA2_Peak1	HyperDMR	2940
ANP_anp-dg	HypoDMR 	24950
ANP_anp-dg	HyperDMR	4856
OLF_Xkr6	HypoDMR 	11033
OLF_Xkr6	HyperDMR	2669
D1L-Fstl4_Cadm1	HypoDMR 	10209
D1L-Fstl4_Cadm1	HyperDMR	6943
IT-L4_Shc3	HypoDMR 	24455
IT-L4_Shc3	HyperDMR	1356
D1L-PAL_Plcxd3	HypoDMR 	7541
D1L-PAL_Plcxd3	HyperDMR	11076
L6b_Pkhd1	HypoDMR 	5175
L6b_Pkhd1	HyperDMR	848


In [15]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    for sample, hypo_index in hypo_sig_dict.items():
        hdf[f'HypoDMR/{sample}'] = pd.Series(hypo_index)
    for sample, hyper_index in hyper_sig_dict.items():
        hdf[f'HyperDMR/{sample}'] = pd.Series(hyper_index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CT-L6_Il1rap'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'NP-L6_Cntnap5a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CGE-Lamp5_Sorcs1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to us

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23_Foxp1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'MGC_mgc-all'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'LSX-Inh_Nxph1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natu

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'MGE-Sst_Ubtd1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'PT-L5_Plcb4'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'MGE-Pvalb_Thsd7a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use 

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'ANP_anp-olf-cnu'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23_Cux1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CGE-Lamp5_Nrxn3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'ASC_str-hpf'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CGE-Vip_Robo1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CA3-St18_Epha5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use na

## Dump DMR bed

In [16]:
hypo_dir = pathlib.Path(dmr_path).parent / 'HypoDMR'
hypo_dir.mkdir(exist_ok=True)
for sample, hypo_index in hypo_sig_dict.items():
    _bed = dmr_bed.loc[hypo_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hypo_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

In [17]:
hyper_dir = pathlib.Path(dmr_path).parent / 'HyperDMR'
hyper_dir.mkdir(exist_ok=True)
for sample, hyper_index in hyper_sig_dict.items():
    _bed = dmr_bed.loc[hyper_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hyper_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

## DMR hits matrix

In [18]:
sig_dict = hypo_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hypo_dir / 'TotalHits.h5ad')


... storing '#chr' as categorical


In [19]:
sig_dict = hyper_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hyper_dir / 'TotalHits.h5ad')

... storing '#chr' as categorical
